# Fuzzy Matching Optimization for Market Research

Task completed for the JetBrains internship.


In [ ]:
import pandas as pd
from math import floor, ceil
import re

In [ ]:
# Create lists with some variations in company names
list_A = ["International Business Machines", "Amazon Web Services", "Apple"]
list_B = [
    "Facebook", "Microsoft Corp.", "Apple Incorporated",
    "Google LLC", "Meta Inc.", "AWS",
    "Microsoft Co.", "Apple, Inc.", "IBM Corp.",
    "Amazon.com", "Alphabet Inc.", "Meta Platforms",
    "Microsoft", "Amplifon", "I.B.M."
]

In [ ]:
def is_full_abbreviation(nameA, nameB):
    ignore_words = ['corp', 'inc', 'co', 'ltd', 'company', 'limited', 'incorporated', 'llc']  # Add more if needed

    # Normalize the input by removing non-alphabetic characters and converting to upper case
    nameA = re.sub(r'[^a-zA-Z\s]', '', nameA).upper()
    nameB = re.sub(r'[^a-zA-Z\s]', '', nameB).upper()

    words = ''
    abbreviation = ''

    # Tokenize the full name into words
    if len(nameA) < len(nameB):
        words = nameB.split()
        abbreviation = nameA.split()[0]
    else:
        words = nameA.split()
        abbreviation = nameB.split()[0]

    # Filter out ignored words
    filtered_words = [word for word in words if word.lower() not in ignore_words]

    # Extract the first letter of each word in the full name
    initials = ''.join(word[0] for word in filtered_words)

    # Compare the acronym with the extracted initials
    return abbreviation == initials or abbreviation in filtered_words

In [ ]:
# Function to calculate the
# Jaro Similarity of two s
def jaro_distance(company_A, company_B):

    # checking if one name is the abbreviation of another
    is_abbreviation = is_full_abbreviation(company_A, company_B)

    # If the companies are equal
    if (company_A == company_B or is_abbreviation):
        return 1.0

    #if (len(company_A) > len(company_B) and is_full_abbrevitaion(company_B, company_A)) or (len(company_B) > len(company_A) and is_full_abbrevitaion(company_A, company_B)):
        #return 1.0

    # Length of two strings
    lenA = len(company_A)
    lenB = len(company_B)

    # Maximum distance upto which matching is allowed
    max_dist = floor(max(lenA, lenB) / 2) - 1

    # Count of matching characters
    matches = 0

    # Hash for matches - arrays to store the matches
    hash_A = [0] * lenA
    hash_B = [0] * lenB

    # Traverse through the first
    for i in range(lenA):
        start = i - max_dist if i > max_dist else 0
        end = i + max_dist + 1 if i + max_dist + 1 < lenB else lenB

        # Check if there is any match
        for j in range(start, end):

            # If there is a match
            if (company_A[i] == company_B[j] and hash_B[j] == 0):
                hash_A[i] = 1
                hash_B[j] = 1
                matches += 1
                break

    # If there is no match
    if (matches == 0):
        return 0.0

    # Number of transpositions
    t = 0
    point = 0

    # Count number of occurrences where two characters match but
    # there is a third matched character in between the indices
    for i in range(lenA):
        if (hash_A[i]):

            # Find the next matched character in second hash
            while (hash_B[point] == 0):
                point += 1

            if (company_A[i] != company_B[point]):
                t += 1
            point += 1
    t = t // 2

    # Return the Jaro Similarity
    return (matches / lenA + matches / lenB +
            (matches - t) / matches) / 3.0

# Jaro Winkler Similarity
def jaro_Winkler(s1, s2) :

    jaro_dist = jaro_distance(s1, s2);

    # If the jaro Similarity is above a threshold
    if (jaro_dist > 0.6) :

        # Find the length of common prefix
        prefix = 0;

        for i in range(min(len(s1), len(s2))) :

            # If the characters match
            if (s1[i] == s2[i]):
                prefix += 1;

            # Else break
            else :
                break;

        # Maximum of 4 characters are allowed in prefix
        prefix = min(4, prefix);

        # Calculate jaro winkler Similarity
        jaro_dist += 0.1 * prefix * (1 - jaro_dist);

    return jaro_dist;

In [ ]:
# Function to get the top 3 closest matches for each company in List A using Jaro distance
def find_best_match(list_A, list_B):
    results_jaro = {}
    results_j_w = {}

    for company_A in list_A:
        similarities_jaro = []
        similarities_jaro_winkler = []

        for company_B in list_B:
            similarity_jaro = jaro_distance(company_A, company_B)
            similarity_j_w = jaro_Winkler(company_A, company_B)

            similarities_jaro.append((company_B, similarity_jaro))
            similarities_jaro_winkler.append((company_B, similarity_j_w))
        print(similarities_jaro)
        print(similarities_jaro_winkler)

        # Sort based on similarity score in descending order and select top 3
        top_matches_jaro = sorted(similarities_jaro, key=lambda x: x[1], reverse=True)[:3]
        results_jaro[company_A] = top_matches_jaro
        top_matches_jw = sorted(similarities_jaro_winkler, key=lambda x: x[1], reverse=True)[:3]
        results_j_w[company_A] = top_matches_jw

    return results_jaro, results_j_w

In [ ]:
# Get the top 3 closest matches for each company in List A using Levenshtein distance
jaro_results, jw_results = find_best_match(list_A, list_B)
# Display results in a DataFrame for clarity
df_results = pd.DataFrame([(key, match_j[0], match_j[1]) for key, matches in jaro_results.items() for match_j in matches],
                                      columns=["Company A", "Match from B - Jaro", "Similarity Score - Jaro-Winkler"])

df_results

[('Facebook', 0.37948028673835127), ('Microsoft Corp.', 0.42007168458781363), ('Apple Incorporated', 0.4927120669056153), ('Google LLC', 0.42634408602150536), ('Meta Inc.', 0.4534050179211469), ('AWS', 0.0), ('Microsoft Co.', 0.44058450510063407), ('Apple, Inc.', 0.40527859237536656), ('IBM Corp.', 1.0), ('Amazon.com', 0.35448028673835125), ('Alphabet Inc.', 0.49948304383788256), ('Meta Platforms', 0.5456221198156682), ('Microsoft', 0.43894862604540025), ('Amplifon', 0.4596774193548387), ('I.B.M.', 1.0)]
[('Facebook', 0.37948028673835127), ('Microsoft Corp.', 0.42007168458781363), ('Apple Incorporated', 0.4927120669056153), ('Google LLC', 0.42634408602150536), ('Meta Inc.', 0.4534050179211469), ('AWS', 0.0), ('Microsoft Co.', 0.44058450510063407), ('Apple, Inc.', 0.40527859237536656), ('IBM Corp.', 1.0), ('Amazon.com', 0.35448028673835125), ('Alphabet Inc.', 0.49948304383788256), ('Meta Platforms', 0.5456221198156682), ('Microsoft', 0.43894862604540025), ('Amplifon', 0.4596774193548387

,Company A,Match from B - Jaro,Similarity Score - Jaro-Winkler
0,International Business Machines,IBM Corp.,1.000000
1,International Business Machines,I.B.M.,1.000000
2,International Business Machines,Meta Platforms,0.545622
3,Amazon Web Services,AWS,1.000000
4,Amazon Web Services,Amazon.com,0.638596
5,Amazon Web Services,Amplifon,0.570175
6,Apple,Apple Incorporated,1.000000
7,Apple,"Apple, Inc.",1.000000
8,Apple,Amplifon,0.658333


In [ ]:
acronym = "IBM Corp."
full_name = "International Business Machines"

if is_full_abbrevitaion(full_name, acronym):
    print(f"The acronym '{acronym}' matches the full name '{full_name}'.")
else:
    print(f"The acronym '{acronym}' does not match the full name '{full_name}'.")


The acronym 'IBM Corp.' matches the full name 'International Business Machines'.
